In [1]:
import re
import yaml

In [2]:
file = '../Data/CountryInfo.140728.txt'

with open(file) as f:
    text = ''.join(f.readlines())
    
text = re.findall('<!--.+-->(.+)', text, flags=re.DOTALL)[0]

In [3]:
def find_tag_values(text, tag):
    text = text.split(f'<{tag}>')
    text = [x.split(f'</{tag}>')[0] for x in text[1:]]
    return text

def find_codes(text, tag):
    values = find_tag_values(text, tag)
    codes = []
    for c in values:
        if '[' in c:
            c = re.findall('\[([A-Z]+) ?', c)
        else:
            c = [c]
        codes.extend(c)
    return codes

In [4]:
countries = find_tag_values(text, 'Country')

In [5]:
entries = []
for country in countries:
    name = find_tag_values(country, 'CountryName')
    if name:
        assert len(name) == 1
        name = name[0]
    
    iso3 = find_codes(country, 'ISO3166-alpha3')
    codes = find_codes(country, 'CountryCode')
    codes.extend(find_codes(country, 'COW-Alpha'))
    codes.extend(find_codes(country, 'FIPS-10'))
    codes.extend(find_codes(country, 'ISO3166-alpha2'))
    codes.extend(iso3)
    
    entries.append({
        'name': name,
        'iso3': iso3,
        'codes': list(set(codes).difference({'---', '--'}))
    })

In [6]:
with open('../Data/Countries.yml', 'w') as f:
    yaml.dump(entries, f)

In [7]:
len(countries)

242